In [1]:
!rm -rf /kaggle/working/*

In [2]:
import gc
import os
import joblib
import random
import warnings
import itertools
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from itertools import combinations
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import LabelEncoder
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold, train_test_split

def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    return df1

def read_preprocess_data():
    train = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
  
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    for col in train_num_agg:
        for col_2 in ['first', 'mean']:
            if 'last' in col and col.replace('last', col_2) in train_num_agg:
                train_num_agg[col.replace('last', col_2) + '_last_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', col_2)]
                train_num_agg[col.replace('last', col_2) + '_last_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', col_2)]

    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)

    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)

    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    
    del train_num_agg, train_cat_agg, train_diff
    cat_features_last = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features_last:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        joblib.dump(encoder, f'{cat_col}_encoder.pkl')

    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
    train.to_parquet('train_fe_v3_loaded_rectified.parquet')
    del train
    gc.collect()
    test = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/test.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)

   
    for col in test_num_agg:
        for col_2 in ['first', 'mean']:
            if 'last' in col and col.replace('last', col_2) in test_num_agg:
                test_num_agg[col.replace('last', col_2) + '_last_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', col_2)]
                test_num_agg[col.replace('last', col_2) + '_last_lag_div'] = test_num_agg[col] / test_num_agg[col.replace('last', col_2)]
    
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)

    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)

    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
 
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
 
    cat_features_last = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features_last:
        loaded_encoder = joblib.load(f'{cat_col}_encoder.pkl')
        test[cat_col] = loaded_encoder.transform(test[cat_col])
   
    num_cols = list(test.dtypes[(test.dtypes == 'float32') | (test.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        test[col + '_round2'] = test[col].round(2)
    test.to_parquet('test_fe_v3_loaded_rectified.parquet')
    

read_preprocess_data()

Starting training feature engineer...


100%|██████████| 458913/458913 [11:51<00:00, 644.66it/s]


Starting test feature engineer...


100%|██████████| 924621/924621 [24:34<00:00, 626.87it/s]
